In [3]:
%pip install -U tf_keras # Keras 2
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#'
You should consider upgrading via the 'c:\Users\User\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [4]:
%pip install tensorflow==2.16.1

^C
Note: you may need to restart the kernel to use updated packages.


ERROR: Exception:
Traceback (most recent call last):
  File "c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\pip\_vendor\urllib3\response.py", line 519, in read
    data = self._fp.read(amt) if not fp_closed else b""
  File "c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 90, in read
    data = self.__fp.read(amt)
  File "c:\Users\User\AppData\Local\Programs\Python\Python310\lib\http\client.py", line 465, in read
    s = self.fp.read(amt)
  File "c:\Users\User\AppData\Local\Programs\Python\Python310\lib\socket.py", line 705, in readinto
    return self._sock.recv_into(b)
  File "c:\Users\User\AppData\Local\Programs\Python\Python310\lib\ssl.py", line 1273, in recv_into
    return self.read(nbytes, buffer)
  File "c:\Users\User\AppData\Local\Pro

     ---------------------                224.1/376.9 MB 232.3 kB/s eta 0:10:58


In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [ ]:
def create_gru_model():
    model = Sequential()

    # Add a GRU layer with 64 units
    model.add(GRU(units=64, input_shape=(None, 1), return_sequences=True))
    model.add(Dropout(0.2))  # Add dropout for regularization

    # Add another GRU layer with 32 units
    model.add(GRU(units=32, return_sequences=False))
    model.add(Dropout(0.2))  # Add dropout for regularization

    # Add a Dense layer to output a single continuous value (next heart rate prediction)
    model.add(Dense(units=1))

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='mean_squared_error',  # Use mean squared error for regression
                  metrics=['mean_absolute_error'])

    return model 

In [ ]:
# Simulate some heart rate data with variable-length sequences
num_samples = 1000
max_timesteps = 20  # Maximum sequence length
features = 1        # Only one feature (heart rate)

# Generate realistic heart rate data with some anomalies
def generate_heart_rate_data(num_samples, max_timesteps):
    x_train = []
    y_train = []
    for _ in range(num_samples):
        length = np.random.randint(5, max_timesteps + 1)
        heart_rate = np.random.normal(70, 10, length)  # Normal heart rate around 70 bpm
        if np.random.rand() > 0.9:  # 10% chance of anomaly
            heart_rate[np.random.randint(0, length)] = np.random.normal(150, 20)  # Anomaly with high heart rate
            y_train.append(1)
        else:
            y_train.append(0)
        x_train.append(heart_rate.reshape(-1, 1))
    return x_train, np.array(y_train).reshape(-1, 1)

x_train, y_train = generate_heart_rate_data(num_samples, max_timesteps)

# Pad sequences to the same length for batching
x_train_padded = pad_sequences(x_train, maxlen=max_timesteps, dtype='float32', padding='post', value=0.0)

# Normalize the data (optional but recommended)
mean = np.mean(x_train_padded)
std = np.std(x_train_padded)
x_train_padded = (x_train_padded - mean) / std

In [ ]:
# Initialize the model (no need to specify input shape since it's variable-length)
model = create_gru_model()
model.summary()

# Train the model
model.fit(x_train_padded, y_train, epochs=10, batch_size=32)

# Step 7: Save the model
model.save("gru_model.h5")